<h1 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Amazon Nova
</h1>


In [ ]:
# # %%capture
# !pip install matplotlib transformers datasets accelerate sentence-transformers
# !pip install ipywidgets

Amazon Nova is a new generation of multimodal understanding and creative content generation models that offer state-of-the-art quality, unparalleled customization, and the best price-performance. Amazon Nova models incorporate the same secure-by-design approach as all AWS services, with built-in controls for the safe and responsible use of AI.

Amazon Nova has two categories of models: 
 - **Understanding models** —These models are capable of reasoning over several input modalities, including text, video, and image, and output text. 
- **Creative Content Generation models** —These models generate images or videos based on a text or image prompt.
  
![imgs/model_intro.png](imgs/nova_intro.png)

**Multimodal Understanding Models**
- **Amazon Nova Micro**: Lightening fast, cost-effective text-only model
- **Amazon Nova Lite**: Fastest, most affordable multimodal FM in the industry for its intelligence tier
- **Amazon Nova Pro**:  The fastest, most cost-effective, state-of-the-art multimodal model in the industry

**Creative Content Generation Models**
- **Amazon Nova Canvas**: State-of-the-art image generation model
- **Amazon Nova Reel**: State-of-the-art video generation model

The following notebooks will be focused primarily on Amazon Nova Understanding Models. 

**Amazon Nova Multimodal understanding** foundation models (FMs) are a family of models that are capable of reasoning over several input modalities, including text, video, documents and/or images, and output text.


<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Setup
</h2>


**Gain Access to the Model**: If you have not yet requested for model access in Bedrock, you do so [request access following these instructions](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html).

![imgs/model_access.png](imgs/model_access.png)

<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  When to Use What?
</h2>


### When to Use `Amazon Nova Micro` 

Amazon Nova Micro (Text Input Only) is the fastest and most affordable option, optimized for large-scale, latency-sensitive deployments like conversational interfaces, chats, and high-volume tasks, such as classification, routing, entity extraction, and document summarization.

### When to Use `Amazon Nova Lite` 

Amazon Nova Lite balances intelligence, latency, and cost-effectiveness. It’s optimized for complex scenarios where low latency (minimal delay) is crucial, such as interactive agents that need to orchestrate multiple tool calls simultaneously. Amazon Nova Lite supports image, video, and text inputs and outputs text. 

### When to Use `Amazon Nova Pro` 
Amazon Nova Pro is designed for highly complex use cases requiring advanced reasoning, creativity, and code generation. Amazon Nova pro supports image, video, and text inputs and outputs text. 

### Prerequisites

Run the cells in this section to install the packages needed by the notebooks in this workshop. ⚠️ You will see pip dependency errors, you can safely ignore these errors.

<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Text Understanding
</h2>


In [1]:
import boto3
import json
import base64

PRO_MODEL_ID = "amazon.nova-pro-v1:0"
LITE_MODEL_ID = "amazon.nova-lite-v1:0"
MICRO_MODEL_ID = "amazon.nova-micro-v1:0"

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime")


### `InvokeModel` body and output

The invoke_model() method of the Amazon Bedrock runtime client (InvokeModel API) will be the primary method we use for most of our Text Generation and Processing tasks

Although the method is shared, the format of input and output varies depending on the foundation model used - as described below:


```python
{
  "system": [
    {
      "text": string
    }
  ],
  "messages": [
    {
      "role": "user",# first turn should always be the user turn
      "content": [
        {
          "text": string
        },
        {
          "image": {
            "format": "jpeg"| "png" | "gif" | "webp",
            "source": {
              "bytes": "base64EncodedImageDataHere..."#  base64-encoded binary
            }
          }
        },
        {
          "video": {
            "format": "mkv" | "mov" | "mp4" | "webm" | "three_gp" | "flv" | "mpeg" | "mpg" | "wmv",
            "source": {
            # source can be s3 location of base64 bytes based on size of input file. 
               "s3Location": {
                "uri": string, #  example: s3://my-bucket/object-key
                "bucketOwner": string #  (Optional) example: 123456789012)
               }
              "bytes": "base64EncodedImageDataHere..." #  base64-encoded binary
            }
          }
        },
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "text": string # prefilling assistant turn
        }
      ]
    }
  ],
 "inferenceConfig":{ # all Optional
    "max_new_tokens": int, #  greater than 0, equal or less than 5k (default: dynamic*)
    "temperature": float, # greater then 0 and less than 1.0 (default: 0.7)
    "top_p": float, #  greater than 0, equal or less than 1.0 (default: 0.9)
    "top_k": int #  0 or greater (default: 50)
    "stopSequences": [string]
  },
  "toolConfig": { #  all Optional
        "tools": [
                {
                    "toolSpec": {
                        "name": string # menaingful tool name (Max char: 64)
                        "description": string # meaningful description of the tool
                        "inputSchema": {
                            "json": { # The JSON schema for the tool. For more information, see JSON Schema Reference
                                "type": "object",
                                "properties": {
                                    <args>: { # arguments 
                                        "type": string, # argument data type
                                        "description": string # meaningful description
                                    }
                                },
                                "required": [
                                    string # args
                                ]
                            }
                        }
                    }
                }
            ],
   "toolChoice": "any" //Amazon Nova models ONLY support tool choice of "any"
        }
    }
}
```

The following are required parameters.

* `system` – (Optional) The system prompt for the request.
    A system prompt is a way of providing context and instructions to Amazon Nova, such as specifying a particular goal or role.
* `messages` – (Required) The input messages.
    * `role` – The role of the conversation turn. Valid values are user and assistant. 
    * `content` – (required) The content of the conversation turn.
        * `type` – (required) The type of the content. Valid values are image, text. , video
            * if chosen text (text content)
                * `text` - The content of the conversation turn. 
            * If chosen Image (image content)
                * `source` – (required) The base64 encoded image bytes for the image.
                * `format` – (required) The type of the image. You can specify the following image formats. 
                    * `jpeg`
                    * `png`
                    * `webp`
                    * `gif`
            * If chosen video: (video content)
                * `source` – (required) The base64 encoded image bytes for the video or S3 URI and bucket owner as shown in the above schema
                * `format` – (required) The type of the video. You can specify the following video formats. 
                    * `mkv`
                    *  `mov`  
                    *  `mp4`
                    *  `webm`
                    *  `three_gp`
                    *  `flv`  
                    *  `mpeg`  
                    *  `mpg`
                    *  `wmv`
* `inferenceConfig`: These are inference config values that can be passed in inference.
    * `max_new_tokens` – (Optional) The maximum number of tokens to generate before stopping.
        Note that Amazon Nova models might stop generating tokens before reaching the value of max_tokens. Maximum New Tokens value allowed is 5K.
    * `temperature` – (Optional) The amount of randomness injected into the response.
    * `top_p` – (Optional) Use nucleus sampling. Amazon Nova computes the cumulative distribution over all the options for each subsequent token in decreasing probability order and cuts it off once it reaches a particular probability specified by top_p. You should alter either temperature or top_p, but not both.
    * `top_k` – (Optional) Only sample from the top K options for each subsequent token. Use top_k to remove long tail low probability responses.
    * `stopSequences` – (Optional) Array of strings containing step sequences. If the model generates any of those strings, generation will stop and response is returned up until that point. 
    * `toolConfig` – (Optional) JSON object following ToolConfig schema,  containing the tool specification and tool choice. This schema is the same followed by the Converse API




<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Synchronous API Call
</h2>

In [2]:
# Define your system prompt(s).
system_list = [
    { "text": "You should respond to all messages in french" }
]

# Define one or more messages using the "user" and "assistant" roles.
message_list = [
    {"role": "user", "content": [{"text": "FreeCodeCamp is a great platform to learn coding and technology in the field of computer science"}]},
]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 300, "top_p": 0.9, "top_k": 20}

native_request = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

# Invoke the model and extract the response body.
response = client.invoke_model(modelId=LITE_MODEL_ID, body=json.dumps(native_request))
request_id = response["ResponseMetadata"]["RequestId"]
model_response = json.loads(response["body"].read())

# # Pretty print the response JSON.
# print("\n[Full Response]")
# print(json.dumps(model_response, indent=2))

# Print the text content for easy readability.
content_text = model_response["output"]["message"]["content"][0]["text"]
print(content_text)

FreeCodeCamp est une excellente plateforme pour apprendre le codage et la technologie dans le domaine des sciences informatiques. Elle offre une variété de ressources éducatives, de tutoriels et de projets pratiques qui sont idéaux pour les débutants comme pour les développeurs expérimentés. Que vous soyez intéressé par le développement web, la science des données, ou d'autres domaines connexes, FreeCodeCamp propose des cours structurés et interactifs pour vous aider à progresser. De plus, la communauté active de FreeCodeCamp fournit un soutien et des opportunités de réseautage précieux. C'est une ressource incroyable pour quiconque souhaite acquérir de nouvelles compétences ou approfondir celles qu'il possède déjà.


<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Streaming API Call
</h2>

In [3]:
from datetime import datetime

# Define your system prompt(s).
system_list = [
    { "text": "Act as a creative writing assistant. When the user provides you with a topic, write a short story about that topic." }
]

# Define one or more messages using the "user" and "assistant" roles.
message_list = [{"role": "user", "content": [{"text": "A camping trip"}]}]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 1000, "top_p": 0.9, "top_k": 20}

request_body = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

start_time = datetime.now()

# Invoke the model with the response stream
response = client.invoke_model_with_response_stream(
    modelId=LITE_MODEL_ID, body=json.dumps(request_body)
)

chunk_count = 0
time_to_first_token = None

# Process the response stream
stream = response.get("body")
if stream:
    for event in stream:
        chunk = event.get("chunk")
        if chunk:
            # Print the response chunk
            chunk_json = json.loads(chunk.get("bytes").decode())
            # Pretty print JSON
            # print(json.dumps(chunk_json, indent=2, ensure_ascii=False))
            content_block_delta = chunk_json.get("contentBlockDelta")
            if content_block_delta:
                if time_to_first_token is None:
                    time_to_first_token = datetime.now() - start_time
                    print(f"Time to first token: {time_to_first_token}")

                chunk_count += 1
                current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S:%f")
                # print(f"{current_time} - ", end="")
                print(content_block_delta.get("delta").get("text"), end="")
    print(f"\nTotal chunks: {chunk_count}")
else:
    print("No response stream received.")

Time to first token: 0:00:00.209212
The sun had just begun its descent beyond the horizon, casting an amber glow over the dense forest. The air was filled with the scent of pine and earth, and a cool breeze rustled the leaves, whispering secrets of the woods. Emma, Jake, and their dog, Max, stood at the edge of a clearing, their backpacks heavy with gear and anticipation.

They had spent the day hiking, the trail winding like a silver ribbon through the towering trees. With each step, they felt the weight of their worries lift, replaced by the simple joy of being in nature. The laughter and banter that had filled the air were now replaced by a comfortable silence, each lost in their thoughts.

As they reached the designated campsite, Emma began to set up the tent, her movements swift and practiced. Jake gathered firewood, stacking the branches with care while Max bounded around, exploring every nook and cranny. The fire crackled to life, its warmth a welcome contrast to the evening chi

<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Image Understanding 
</h2>


Amazon Nova models allow you to include multiple images in the payload with a limitation of total payload size to not go beyond 25MB. 
Amazon Nova models can analyze the passed images and answer questions, classify an image, as well as summarize images based on provided instructions.

## Image size information

To provide the best possible results, Amazon Nova automatically rescales input images up or down depending on their aspect ratio and original resolution. For each image, Amazon Nova first identifies the closest aspect ratio from 1:1, 1:2, 1:3, 1:4, 1:5, 1:6, 1:7, 1:8, 1:9 2:3, 2:4 and their transposes. Then the image is rescaled so that at least one side of the image is greater than 896px or the length of shorter side of the original image, while maintaining the closest aspect ratio. There's a maximum resolution of 8,000x8,000 pixels.

## Image to tokens conversion

As previously discussed, images are resized to maximize information extraction, while still maintaining the aspect ratio. What follows are some examples of sample image dimensions and approximate token calculations.

| image_resolution (HxW or WxH) | 900 x 450 | 900 x 900 | 1400 x 900 | 1.8K x 900 | 1.3Kx1.3K |
|------------------------------|-----------|-----------|------------|------------|-----------|
| Estimated token count        | ~800      | ~1300     | ~1800      | ~2400      | ~2600     |

So for example, consider an example image that is 800x400 in size, and you want to estimate the token count for this image. Based on the dimensions, to maintain an aspect ratio of 1:2, the closest resolution is 900x450. Therefore, the approximate token count for this image is about 800 tokens.


Lets see how Nova model does on Image Understanding Usecase. 

Here we will pass this image and ask model to try to create 3 art titles for this image. 

![Snow](imgs/snow.png)

In [ ]:


# Open the image you'd like to use and encode it as a Base64 string.
with open("imgs/snow.png", "rb") as image_file:
    binary_data = image_file.read()
    base_64_encoded_data = base64.b64encode(binary_data)
    base64_string = base_64_encoded_data.decode("utf-8")

# Define your system prompt(s).
system_list = [
    { "text": "You are an expert artist. When the user provides you with an image, provide 3 potential art titles" }
]

# Define a "user" message including both the image and a text prompt.
message_list = [
    {
        "role": "user",
        "content": [
            {
                "image": {
                    "format": "png",
                    "source": {"bytes": base64_string},
                }
            },
            {"text": "Provide art titles for this image."},
        ],
    }
]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 300, "top_p": 0.1, "top_k": 20}

native_request = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

# Invoke the model and extract the response body.
response = client.invoke_model(modelId=LITE_MODEL_ID, body=json.dumps(native_request))
model_response = json.loads(response["body"].read())

# Pretty print the response JSON.
print("[Full Response]")
print(json.dumps(model_response, indent=2))

# Print the text content for easy readability.
content_text = model_response["output"]["message"]["content"][0]["text"]
print("\n[Response Content Text]")
print(content_text)

There can be multiple image contents. In this example we ask the model to find what two images have in common:

![](imgs/cat.jpeg)

![](imgs/dog.jpeg)

In [ ]:

# Open the image you'd like to use and encode it as a Base64 string.
with open("imgs/dog.jpeg", "rb") as image_file:
    binary_data = image_file.read()
    base_64_encoded_data = base64.b64encode(binary_data)
    dog_base64_string = base_64_encoded_data.decode("utf-8")

with open("imgs/cat.jpeg", "rb") as image_file:
    binary_data = image_file.read()
    base_64_encoded_data = base64.b64encode(binary_data)
    cat_base64_string = base_64_encoded_data.decode("utf-8")

# Define a "user" message including both the image and a text prompt.
message_list = [
    {
        "role": "user",
        "content": [
            {
                "image": {
                    "format": "jpeg",
                    "source": {"bytes": dog_base64_string},
                }
            },
            {
                "image": {
                    "format": "jpeg",
                    "source": {"bytes": cat_base64_string},
                }
            },
            {"text": "What do these two images have in common?"},
        ],
    }
]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 300, "top_p": 0.1, "top_k": 20}

native_request = {
    "messages": message_list,
    "inferenceConfig": inf_params,
}

# Invoke the model and extract the response body.
response = client.invoke_model(modelId=LITE_MODEL_ID, body=json.dumps(native_request))
model_response = json.loads(response["body"].read())

# # Pretty print the response JSON.
# print("[Full Response]")
# print(json.dumps(model_response, indent=2))

# Print the text content for easy readability.
content_text = model_response["output"]["message"]["content"][0]["text"]
print(content_text)

<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Video Understanding
</h2>


The Amazon Nova models allow you to include a single video in the payload, which can be provided either in base64 format or through an Amazon S3 URI. When using the base64 method, the overall payload size must remain within 25MB. However, you can specify an Amazon S3 URI for video understanding. This approach enables you to leverage the model for longer videos (up to 1GB in size) without being constrained by the overall payload size limitation. Amazon Nova models can analyze the passed video and answer questions, classify a video, and summarize information in the video based on provided instructions.

| Media File Type | File Formats supported | Input Method |
|----------------|------------------------|--------------|
| Video | MP4, MOV, MKV, WebM, FLV, MPEG, MPG, WMV, 3GP | Base64 <br> _(Recommended for payload size less than 25MB)_ <br><br> Amazon S3 URI <br> _(Recommended for payload greater than 25MB up to 1GB)_ |

There are no differences in the video input token count, regardless of whether the video is passed as base64 (as long as it fits within the size constraints) or via an Amazon S3 location.

Note that for 3gp file format, the "format" field passed in the API request should be of the format "three_gp".

When using Amazon S3, ensure that you are set the "Content-Type" metadata to the correct MIME type for the video.

## Video size information

Amazon Nova video understanding capabilities support Multi-Aspect Ratio. All videos are resized with distortion (up or down, based on the input) to 672*672 square dimensions before feeding it to the model. The model utilizes a dynamic sampling strategy based on the length of the video. For videos less than or equal to 16 minutes in duration, a 1 frame per second (FPS) sampling rate is employed. However, for videos exceeding 16 minutes in length, the sampling rate decreases in order to maintain a consistent 960 frames sampled, with the frame sampling rate varying accordingly. This approach is designed to provide more accurate scene-level video understanding for shorter videos compared to longer video content. We recommend that you keep the video length less than 1 hour for low motion, and less than 16 minutes for anything with higher motion.

There should be no difference when analyzing a 4k version of a video and a Full HD version. Similarly, because the sampling rate is at most 1 FPS, a 60 FPS video should perform as well as a 30 FPS video. Because of the 1GB limit in video size, using higher than required resolution and FPS is not beneficial and will limit the video length that fits in that size limit. You might want to pre-process videos longer than 1GB.

## Video tokens

The length of the video is main factor impacting the number of tokens generated. To calculate the approximate cost, you should multiply the estimated number of video tokens by the per-token price of the specific model being utilized.

This table has some approximations of frame sampling and token utilization per video length:

| video_duration | 10 sec | 30 sec | 16 min | 20 min | 30 min | 45 min | 1 hr | 1.5 hr |
|----------------|---------|---------|---------|---------|---------|---------|-------|---------|
| frames_to_sample | 10 | 30 | 960 | 960 | 960 | 960 | 960 | 960 |
| sample_rate_fps | 1 | 1 | 1 | 0.755 | 0.5 | 0.35556 | 0.14 | 0.07 |
| Estimated token count | 2,880 | 8,640 | 276,480 | 276,480 | 276,480 | 276,480 | 276,480 | 276,480 |


In [4]:
from IPython.display import Video

Video("imgs/the-sea.mp4")

In [8]:
# Open the image you'd like to use and encode it as a Base64 string.
with open("imgs/the-sea.mp4", "rb") as video_file:
    binary_data = video_file.read()
    base_64_encoded_data = base64.b64encode(binary_data)
    base64_string = base_64_encoded_data.decode("utf-8")

# Define your system prompt(s).
system_list = [
    { "text": "You are an expert media analyst. When the user provides you with a video, provide 3 potential video titles" }
]

# Define a "user" message including both the image and a text prompt.
message_list = [
    {
        "role": "user",
        "content": [
            {
                "video": {
                    "format": "mp4",
                    "source": {"bytes": base64_string},
                }
            },
            {"text": "Provide video titles for this clip."},
        ],
    }
]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 300, "top_p": 0.1, "top_k": 20}

native_request = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

# Invoke the model and extract the response body.
response = client.invoke_model(modelId=LITE_MODEL_ID, body=json.dumps(native_request))
model_response = json.loads(response["body"].read())

# # Pretty print the response JSON.
# print("[Full Response]")
# print(json.dumps(model_response, indent=2))

# Print the text content for easy readability.
content_text = model_response["output"]["message"]["content"][0]["text"]
print("\n[Response Content Text]")
print(content_text)


[Response Content Text]
1. "Majestic Ocean Cliffs and a Shell's Journey: A Coastal Adventure"
2. "From Rocky Shores to Sandy Beaches: A Coastal Exploration"
3. "Waves, Rocks, and a Shell's Tale: A Scenic Coastal Journey"


In [12]:
import boto3
import base64
import json

def send_to_amazon_nova_pro(video_path: str) -> dict:
    """
    Sends a video file to the Amazon Nova Pro model and returns the model's response.

    Args:
        video_path (str): The file path to the video.

    Returns:
        dict: The response from the Amazon Nova Pro model.
    """
    # Initialize the Bedrock Runtime client
    client = boto3.client("bedrock-runtime", region_name="us-east-1")

    # Read and encode the video file
    with open(video_path, "rb") as video_file:
        binary_data = video_file.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode("utf-8")


    # Define the system prompt
    system_list  = [
        {
            "text": "You are an expert media analyst. Analyze the provided video and generate a detailed description, "
                    "including identification of important elements such as persons, vehicles, road signs, and timestamps. "
                    "Assess the severity of any safety incidents observed."
        }
    ]

    # Define the user message with the video content
    message_list = [
        {
            "role": "user",
            "content": [
                {
                    "video": {
                        "format": "mp4",
                        "source": {"bytes": base64_string},
                    }
                },
                {"text": "Analyze this video and provide a detailed report."},
            ],
        }
    ]

    # Configure inference parameters
    inf_params = {"max_new_tokens": 300, "top_p": 0.1, "top_k": 20}


    # Construct the request payload
    native_request = {
                        "messages": message_list,
                        "system": system_list,
                        "inferenceConfig": inf_params,
                        }

    # Invoke the Amazon Nova Pro model
    response = client.invoke_model(modelId=LITE_MODEL_ID, body=json.dumps(native_request))
    model_response = json.loads(response["body"].read())

    # Parse and return the model's response
    content_text = model_response["output"]["message"]["content"][0]["text"]

    return content_text


In [13]:
analysis_result = send_to_amazon_nova_pro("imgs/the-sea.mp4")

In [47]:
from concurrent.futures import ThreadPoolExecutor

# List of video paths to be processed
video_paths = ["video1.mp4", "video2.mp4", "video3.mp4"]

# Process videos in parallel using ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    results = list(executor.map(send_to_amazon_nova_pro, video_paths))

# Print results
for i, analysis in enumerate(results):
    print(f"\n[Analysis Result for Video {i+1}]")
    print(analysis)


[Analysis Result for Video 1]
The video starts with a top-down view of a rocky coastline with waves crashing against the rocks. The camera then zooms in on a large shell on the beach, with the waves gently washing over it. The shell is a spiral-shaped seashell with a brown and white pattern. The sand is wet and has small footprints on it. The sun is shining brightly, and the light is reflecting on the water. The video ends with a close-up of the shell, showing its intricate design and the way the light is reflecting off it.

[Analysis Result for Video 2]
The video starts with a top-down view of a rocky coastline with waves crashing against the rocks. The camera then zooms in on a large shell on the beach, with the waves gently washing over it. The shell is a spiral-shaped seashell with a brown and white pattern. The sand is wet and has small footprints on it. The sun is shining brightly, and the light is reflecting on the water. The video ends with a close-up of the shell, showing its

In [42]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

# List of video paths to be processed
video_paths = ["imgs/the-sea-small.mp4", "imgs/the-sea-small.mp4", "imgs/the-sea-small.mp4"]

# Create a RunnableLambda for the send_to_amazon_nova_pro function
runnable1 = RunnableLambda(send_to_amazon_nova_pro)
runnable2 = RunnableLambda(send_to_amazon_nova_pro)
runnable3 = RunnableLambda(send_to_amazon_nova_pro)

sequence = runnable1 | runnable2 | runnable3




### Video Understanding using S3 Path

In [ ]:

# Define your system prompt(s).
system_list = [
    { "text": "You are an expert media analyst. When the user provides you with a video, provide 3 potential video titles" }
]

# Define a "user" message including both the image and a text prompt.
message_list = [
    {
        "role": "user",
        "content": [
            {
                "video": {
                    "format": "mp4",
                    "source": {
                        "s3Location": {
                            # Replace the S3 URI
                            "uri": "s3://my-bedrock-dataset-1/the-sea.mp4"
                        }
                    },
                }
            },
            {"text": "Provide video titles for this clip."},
        ],
    }
]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 300, "top_p": 0.1, "top_k": 20, "temperature": 0.3}

native_request = {
    "schemaVersion": "messages-v1",
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

# Invoke the model and extract the response body.
response = client.invoke_model(modelId=PRO_MODEL_ID, body=json.dumps(native_request))
model_response = json.loads(response["body"].read())

# # Pretty print the response JSON.
# print("[Full Response]")
# print(json.dumps(model_response, indent=2))

# Print the text content for easy readability.
content_text = model_response["output"]["message"]["content"][0]["text"]
print(content_text)